# Le plan des villes schématisé

Fork des [travaux de gboeing](https://github.com/gboeing/osmnx-examples/blob/master/notebooks/17-street-network-orientations.ipynb)

Compare the spatial orientations of city street networks with OSMnx.

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [67]:
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

ox.config(log_console=True, use_cache=True)
weight_by_length = False

In [ ]:
# Définir le nom des villes souhaitées (pour le Québec)
places = {'Montréal'       : 'Montreal, Canada',
          'Québec'       : 'Quebec city, Quebec, Canada',
          'Laval'       : 'Laval, Canada',
          'Gatineau'       : 'Gatineau, Canada',
          'Longueuil'       : 'Longueuil, Canada',
          'Sherbrooke'       : 'Sherbrooke, Canada',
          'Lévis'       : 'Lévis, Canada',
          #'Saguenay'       : 'Saguenay, Canada', - Indisponible
          #'Trois-Rivières'       : 'Trois-Rivières, Canada', - Indisponible
          'Terrebonne'       : 'Terrebonne, Canada',
          'Saint-Jean-sur-Richelieu'       : 'Saint-Jean-sur-Richelieu, Canada'}

In [79]:
# Définir le nom des villes souhaitées (pour le Canada)
# places = {'Toronto'       : 'Toronto, Canada',
#           'Montréal'       : 'Montreal, Canada',
#           'Calgary'       : 'Calgary, Canada',
#           'Ottawa'       : 'Ottawa, Canada',
#           #'Edmonton'       : 'Edmonton city, Canada',  - Indisponible
#           #'Mississauga'       : 'Mississauga, Ontario, Canada',  - Indisponible
#           'Winnipeg'       : 'Winnipeg, Canada',
#           'Vancouver'       : 'Vancouver, Canada',
#           'Brampton'       : 'Brampton, Canada',
#           #'Hamilton'       : 'Hamilton, Canada',  - Indisponible
#           'Québec'       : 'Quebec city, Quebec, Canada',
#           'Surrey'       : 'Surrey, Canada'}

In [89]:
# verify OSMnx geocodes each query to what you expect
gdf = ox.gdf_from_places(places.values())
gdf

,bbox_east,bbox_north,bbox_south,bbox_west,geometry,place_name
0,-73.473487,45.704790,45.410076,-73.972902,"POLYGON ((-73.97290169999999 45.461403, -73.93...","Montréal, Montréal (06), Québec, Canada"
1,-71.133661,46.980680,46.727710,-71.549217,"POLYGON ((-71.5492175 46.8511815, -71.5313905 ...","Québec, Québec (Agglomération), Capitale-Natio..."
2,-73.523504,45.700749,45.510875,-73.895360,"POLYGON ((-73.8953597 45.5265897, -73.89201749...","Laval (13), Québec, Canada"
3,-75.344372,45.599124,45.372383,-75.908351,"POLYGON ((-75.9083514 45.4804329, -75.90788499...","Gatineau, Outaouais, Québec, Canada"
4,-73.338981,45.588947,45.432983,-73.529187,"POLYGON ((-73.5291874 45.5346018, -73.5277098 ...","Longueuil, Longueuil (agglomeration), Montérég..."
5,-71.802542,45.524050,45.299905,-72.108789,"POLYGON ((-72.1087893 45.3014946, -72.09620649...","Sherbrooke, Québec, Canada"
6,-70.993781,46.842247,46.572183,-71.518922,"POLYGON ((-71.51892170000001 46.6981065, -71.5...","Lévis, Chaudière-Appalaches, Québec, Canada"
7,-73.487080,45.827164,45.668717,-73.844778,"POLYGON ((-73.8447776 45.7209704, -73.8283125 ...","Terrebonne, Les Moulins, Lanaudière, Québec, C..."
8,-73.168023,45.413102,45.233265,-73.411444,"POLYGON ((-73.41144439999999 45.2548764, -73.3...","St-Jean-s-Richelieu, Le Haut-Richelieu, Montér..."


## Get the street networks and their edge bearings

In [90]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

In [91]:
bearings = {}
for place in sorted(places.keys()):
    
    # get the graph
    query = places[place]
    G = ox.graph_from_place(query, network_type='drive')
    
    # calculate edge bearings
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    
    if weight_by_length:
        # weight bearings by length (meters)
        city_bearings = []
        for u, v, k, d in Gu.edges(keys=True, data=True):
            city_bearings.extend([d['bearing']] * int(d['length']))
        b = pd.Series(city_bearings)
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    else:
        # don't weight bearings, just take one value per street segment
        b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

## Visualize it

In [92]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

In [93]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):

    bins = np.arange(n + 1) * 360 / n
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#003366', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'Arial', 'size':24, 'weight':'bold'}
    xtick_font = {'family':'Arial', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'Arial', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'O', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [94]:
# create figure and axes
n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})

# plot each city's polar histogram
for ax, place in zip(axes.flat, sorted(places.keys())):
    polar_plot(ax, bearings[place].dropna(), title=place)

# add super title and save full image
suptitle_font = {'family':'Arial', 'fontsize':60, 'fontweight':'bold', 'y':1.07}
fig.suptitle('9 villes québécoises schématisées', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
fig.savefig('images/street-orientations2.png', dpi=120, bbox_inches='tight')
plt.close()